# *Импорт библиотек*

In [ ]:
%%capture
!pip install deskew

In [ ]:
from google.colab import files # Импортируем Модуль files для работы с файлами
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
%matplotlib inline
import seaborn as sns

plt.style.use('seaborn-white')
sns.set_context('notebook', rc={'figure.figsize': (10, 8)}, font_scale=1.5)

import numpy as np # Импортируем библиотеку numpy
import pandas as pd

import time
import random
import math
import re
import os # Импортируем библиотеку os для раоты с фаловой системой
import shutil
import json
import cv2

from copy import deepcopy
from IPython.display import clear_output

from deskew import determine_skew

In [ ]:
from PIL import ImageFile, ImageOps # импортируем модель Image для работы с изображениями
from PIL import Image as PILImage
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from google.colab import drive # Подключаем гугл-диск
drive.mount('/content/drive')

In [ ]:
# Контекстный менеджер для измерения времени операций
# Операция обертывается менеджером с помощью оператора with
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        # print('Время обработки: {:.2f} с'.format(time.time() - self.t))
        seconds_total = time.time() - self.t
        minutes = int(seconds_total // 60)
        seconds = (seconds_total % 60) + (seconds_total - math.floor(seconds_total))
        print('Время обработки: {} мин {:.2f} c'.format(minutes, seconds))

with timex():
    print()

# **Загрузка датафрейма и составление выборок**

In [ ]:
# Загрузка
df = pd.read_csv('/content/drive/MyDrive/df_info_new_ru.csv')

In [ ]:
df.columns

In [ ]:
df = df.sort_values(by='индекс файла')

In [ ]:
print(*df[df['проверка таблиц'] == 1].filename_new[:10], sep='\n')

In [ ]:
def get_img_path(base_path, filename):
    x = int(filename[:-4])
    lower_bound = (x // 100) * 100
    upper_bound = lower_bound + 100
    return base_path + f'{lower_bound}-{upper_bound}/' + filename

# Проверка
base_path = '/content/drive/MyDrive/ClearedReshapedBase/'
x_values = ['0001.png', '0019.png', '0100.png', '0654.png', '1119.png']

for x in x_values:
    y = get_img_path(base_path, x)
    print(y)

In [ ]:
# Составляем выборки файлов для проверки таблиц
base_path = '/content/drive/MyDrive/ClearedReshapedBase/'
target_cols =  ['проверка таблиц', 'нестандартная верхняя таблица', 'нестандартная нижняя таблица',
                'обрезанный текст', 'обрезанные границы таблиц', 'не белый фон', 'точечный текст',
                'плохая печать', 'искажение текста', 'искажение документа', '2 документа',
                '2 банка в верхней таблице', 'грязь']

sample_dict = {col_name:[get_img_path(base_path, i) for i in list(df[df[col_name] == 1].filename_new)] for col_name in target_cols}

In [ ]:
def print_img(image_path, figsize=(12,8), title=None):
    plt.rcParams['figure.facecolor'] = 'lightgray'
    img = PILImage.open(image_path)
    img = img.convert('RGB')

    # Отображение изображения с помощью Matplotlib
    plt.figure(figsize=figsize)
    plt.imshow(img)
    if title == None:
        plt.title(image_path[-8:])  # Установка заголовка из названия файла
    else:
        plt.title(title)  # Установка заголовка из названия файла
    plt.axis('off')  # Убрать оси координат
    plt.show()

# **to pdf**
> **Проверка библиотек для конвертации в pdf:** PIL, img2pdf, pytesseract

In [ ]:
img_folder = '/content/'

file_list = [i for i in os.listdir(img_folder) if '.png' in i]
file_list

In [ ]:
file_path_list = [img_folder + i for i in file_list]
file_path_list

In [ ]:
# Создание папки
target_folder = '/content/pdf_images/'

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

In [ ]:
# Примеры изображений
for image_path in [img_folder + i for i in file_list]:
    img = Image.open(image_path)

    # Отображение изображения с помощью Matplotlib
    plt.figure(figsize=(12,8))
    plt.imshow(img)
    plt.title(image_path)  # Установка заголовка из названия файла
    plt.axis('off')  # Убрать оси координат
    plt.show()
    print()

### *PIL to pdf*
> Просто вставляет картинку в pdf

In [ ]:
from PIL import Image

for img in file_list:
    image_1 = Image.open(img_folder + img)
    im_1 = image_1.convert('RGB')
    im_1.save(target_folder + img[:-4] + '.pdf')

In [ ]:
pdf_file_list = [target_folder + i for i in os.listdir(target_folder)]
pdf_file_list

## *img2pdf*

In [ ]:
!pip uninstall PIL -y

In [ ]:
%%capture
pip install img2pdf

In [ ]:
# importing necessary libraries
import img2pdf
from PIL import Image
import os

for img in file_list:
    img_path = img_folder + img
    pdf_path = target_folder + img[:-4] + '.pdf'

    image = Image.open(img_path)
    pdf_bytes = img2pdf.convert(image.filename)

    # opening or creating pdf file
    file = open(pdf_path, "wb")

    # writing pdf files with chunks
    file.write(pdf_bytes)

    image.close()
    file.close()

In [ ]:
pdf_file_list = [target_folder + i for i in os.listdir(target_folder)]
pdf_file_list

## *pytesseract*

In [ ]:
!pip install pytesseract
!pip install pdf2image
!apt-get install poppler-utils
!apt-get install tesseract-ocr

In [ ]:
import pytesseract

In [ ]:
for img in file_list:
    img_path = img_folder + img
    pdf_path = target_folder + img[:-4] + '.pdf'

    pdf = pytesseract.image_to_pdf_or_hocr(img_path, extension='pdf', lang='rus+eng')

    with open(pdf_path, 'w+b') as f:
        f.write(pdf) # pdf type is bytes by default

# **table recognition .PDF (pdfplumber, PyMuPDF, tabula, camelot, PyPDF2, pypdf)**
> Результаты для конвертации .png в .pdf через **pytesseract**. По другим методам не находит текста в файле.

> **Библиотеки:** pdfplumber, PyMuPDF, tabula, camelot, PyPDF2, pypdf.

> **Результаты**:
* Из-за низкого качества преобразования файлов через **pytesseract**, не получилось нормально распознать таблицы.
* Лучше всего работает библиотека ***tabula***, из которой можно попробовать вытащить готовые таблицы, если такие были найдены.
* Такой метод, возможно, подойдёт если получится добится хорошего распознования текста с помощью **pytesseract** или других библиотек, которые способны сохранять изображения в .pdf с сохранением текста в файле.

## *pdfplumber*

In [ ]:
%%capture
pip install pdfplumber

In [ ]:
import pdfplumber

for pdf_path in pdf_file_list:
    with pdfplumber.open(pdf_path) as pdf:
        first_page = pdf.pages[0]
        print(f'file: {pdf_path}')
        print('types:')
        print(f'\t objects:\t{first_page.objects}')
        print(f'\t chars: \t{first_page.chars}')
        print(f'\t lines: \t{first_page.lines}')
        print(f'\t rects: \t{first_page.rects}')
        print(f'\t curves:\t{first_page.curves}')
        print(f'\t images:\t{first_page.images}')
        print(pdf.metadata)
        print(first_page.annots)
        print(first_page.bbox)
        print(first_page.cached_properties)
        print(first_page.chars)
        print(first_page.extract_text())
        print(first_page.extract_table())
        print(first_page.find_table(table_settings={}))
        print()
        print('-'*100)

## *PyMuPDF*

In [ ]:
pip install PyMuPDF

In [ ]:
import fitz

for pdf_path in pdf_file_list:
    print(f'file: {pdf_path}')
    doc = fitz.open(pdf_path)
    print(doc.metadata)
    print(doc.get_toc())
    page = doc.load_page(0)

    text = page.get_text()
    areas = page.search_for("mupdf")
    print(text)
    print(areas)
    print()
    print('-'*100)

## *tabula*
> Лучше всего находит и составляет таблицы

In [ ]:
!pip install -q tabula-py

In [ ]:
import tabula

In [ ]:
for pdf_path in pdf_file_list:
    print(f'file: {pdf_path}')
    dfs = tabula.read_pdf(pdf_path, stream=True, pages=1)

    print(len(dfs))
    try:    print(dfs[0])
    except: pass

    print()
    print('-'*100)

## *camelot*
> Не заработал

In [ ]:
!pip uninstall camelot
!pip uninstall camelot-py
!pip install camelot-py[cv]

In [ ]:
!pip uninstall PyPDF2 -y
!pip install 'PyPDF2<3.0'

In [ ]:
!pip install --upgrade PyPDF2==2.12.1

In [ ]:
import camelot

for pdf_path in pdf_file_list:
    print(f'file: {pdf_path}')
    tables = camelot.read_pdf(pdf_path)
    print(tables)
    try:    print(tables[0].parsing_report)
    except: pass

    print()
    print('-'*100)

## *PyPDF2*

In [ ]:
!pip install PyPDF2

In [ ]:
!pip install --upgrade PyPDF2==2.12.1

In [ ]:
from PyPDF2 import PdfReader

for pdf_path in pdf_file_list:
    print(f'file: {pdf_path}')
    reader = PdfReader(pdf_path)
    print(len(reader.pages))
    page = reader.pages[0]
    print(reader.pages[0])
    print('text:\t',page.extract_text())

    print()
    print('-'*100)

## *pypdf*

In [ ]:
pip install pypdf

In [ ]:
from pypdf import PdfReader

for pdf_path in pdf_file_list:
    print(f'file: {pdf_path}')
    reader = PdfReader(pdf_path)
    print(len(reader.pages))
    page = reader.pages[0]
    print(reader.pages[0])
    print('text:\t',page.extract_text())

    print()
    print('-'*100)

# **table recognition .PNG (img2table)**
> **Библиотека**: img2table.

> **Проверка OCR**: EasyOCR, TesseractOCR, PaddleOCR, DocTR.

> **Результаты**:
* img2table неплохо находит боксы таблиц, особенно, если рамки таблиц ровные, без смещения.
* Находит только таблицы, без лишнего текста.
* Находит даже объеденённые ячейки.
* Удобная функция вывода таблиц сразу в датафрейм.
* Плохо работает, если таблицы с отсутствующими рамками, неявными ячейками в таблицах или с дефектами печати (не пропечатанными линиями/текстом).
* Дополнительные настройки библиотеки img2table, а именно borderless_tables (нахождение таблиц без рамок) и implicit_rows (неявные строки в таблицах) не сработали на нашем датасете.
* Иногда распознаёт 2 столбца как один (чаще всего "Ед. Изм" и "Кол-во"), и/или объеденяет их значения.
* Лучше всего распознают текст **EasyOCR** и **TesseractOCR**. Можно попробовать взять настройки предобработки изображений для улучшения результатов распознания текста.
* Достаточно медленная скорость работы. Время на обработки одного изображения (нахождение таблиц и распознание текста в них):
* * **EasyOCR**: 1 мин. 20 сек.
* * **TesseractOCR**: 30 сек.
* Можно вернутся к этой библиотеке, когда у нас будет дообученный OCR.



In [ ]:
!pip install img2table[paddle] -q
!pip install img2table[easyocr] -q

In [ ]:
!pip install python-doctr -q
!pip install "python-doctr[tf]" -q

!pip install pytesseract -q
!apt-get install tesseract-ocr -q
!apt-get install tesseract-ocr-rus -q
!apt-get install tesseract-ocr-kaz -q

!pip install "paddleocr>=2.0.1" -q

In [ ]:
from img2table.ocr import EasyOCR, TesseractOCR, PaddleOCR, DocTR
from img2table.document import Image
from IPython.display import display_html, display

# Создание моделей OCR с настройками
ocr = EasyOCR(lang=['ru',"en"])
tesseract_ocr = TesseractOCR(lang="rus+eng+kaz")
paddle_ocr_cr = PaddleOCR(lang='cyrillic')
paddle_ocr_en = PaddleOCR(lang="en")
doctr = DocTR(detect_language=True, kw={"detect_orientation": True})

In [ ]:
# Фукция для вывода изображения с боксами
def print_img_with_bboxes(image_path, extracted_tables, figsize=(12,8)):
    table_img = cv2.imread(image_path)

    for table in extracted_tables:
        for row in table.content.values():
            for cell in row:
                cv2.rectangle(table_img, (cell.bbox.x1, cell.bbox.y1), (cell.bbox.x2, cell.bbox.y2), (255, 0, 0), 2)

    img = PILImage.fromarray(table_img)
    img = img.convert('RGB')

    plt.rcParams['figure.facecolor'] = 'lightgray'
    plt.figure(figsize=figsize)
    plt.imshow(img)
    plt.title(image_path[-8:])
    plt.axis('off')
    plt.show()

In [ ]:
# Файл для проверки
target_file = sample_dict['проверка таблиц'][0]

In [ ]:
# Открываем документ
doc = Image(target_file)

# Названия и модели OCR
ocr_dict = {
    'EasyOCR':ocr,
    'tesseract':tesseract_ocr,
    'paddle cyrillic': paddle_ocr_cr,
    'paddle eng': paddle_ocr_en,
    'doctr':doctr,
}

# Словарь для результатов OCR
extracted_tables_dict = dict()

# Вытаскиваем таблицы из изображения
for ocr_name, ocr_m in ocr_dict.items():
    print(ocr_name)
    with timex():
        extracted_tables = doc.extract_tables(ocr=ocr_m,
                                            implicit_rows=False,
                                            borderless_tables=False,
                                            min_confidence=50)

        extracted_tables_dict[ocr_name] = extracted_tables
    print()

In [ ]:
print('{:*^165}'.format(' Bounding Boxes '))
print_img_with_bboxes(target_file, extracted_tables_dict['EasyOCR'], (30,15))
for ocr_name, ext in extracted_tables_dict.items():
    print('-'*165)
    print('{:-^165}'.format(' ' + ocr_name + ' '))
    print('-'*165)
    for idx, table in enumerate(ext):
        display_html(table.html_repr(title=f"Таблица {idx + 1}"), raw=True)

**Лучше всего распознают текст EasyOCR и tesseract**

## *confidence=50*

In [ ]:
# Названия и модели OCR
ocr_dict = {
    'EasyOCR':ocr,
    'tesseract':tesseract_ocr,
}

# Словарь для результатов OCR
extracted_tables_files_dict = dict()

for idx, target_file in enumerate(sample_dict['проверка таблиц'][1:30]):
    extracted_tables_dict = dict()

    # Открываем документ
    doc = Image(target_file)

    print(idx)

    # Вытаскиваем таблицы из изображения
    for ocr_name, ocr_m in ocr_dict.items():
        print(ocr_name)
        with timex():
            extracted_tables = doc.extract_tables(ocr=ocr_m,
                                                implicit_rows=False,
                                                borderless_tables=False,
                                                min_confidence=50)

            extracted_tables_dict[ocr_name] = extracted_tables
        print()

    clear_output()

    extracted_tables_files_dict[target_file[-8:]] = extracted_tables_dict

In [ ]:
for filename, extracted_tables_dict in extracted_tables_files_dict.items():
    print('{:*^165}'.format(f' File {filename} | Bounding Boxes '))
    print_img_with_bboxes(get_img_path(base_path, filename), extracted_tables_dict['EasyOCR'], (30,15))
    for ocr_name, ext in extracted_tables_dict.items():
        print('-'*165)
        print('{:-^165}'.format(' ' + ocr_name + ' '))
        print('-'*165)
        for idx, table in enumerate(ext):
            display_html(table.html_repr(title=f"Таблица {idx + 1}"), raw=True)

In [ ]:
# Названия и модели OCR
ocr_dict = {
    'EasyOCR':EasyOCR(lang=['ru',"en"]),
    'tesseract':TesseractOCR(lang="rus+eng+kaz"),
}

# Словарь для результатов OCR
extracted_tables_files_dict = dict()

for idx, target_file in enumerate(sample_dict['нестандартная нижняя таблица'][:10]):
    extracted_tables_dict = dict()

    # Открываем документ
    doc = Image(target_file)

    print(idx)

    # Вытаскиваем таблицы из изображения
    for ocr_name, ocr_m in ocr_dict.items():
        print(ocr_name)
        with timex():
            extracted_tables = doc.extract_tables(ocr=ocr_m,
                                                implicit_rows=False,
                                                borderless_tables=False,
                                                min_confidence=50)

            extracted_tables_dict[ocr_name] = extracted_tables
        print()

    clear_output()

    extracted_tables_files_dict[target_file[-8:]] = extracted_tables_dict

In [ ]:
for filename, extracted_tables_dict in extracted_tables_files_dict.items():
    print('{:*^165}'.format(f' File {filename} | Bounding Boxes '))
    print_img_with_bboxes(get_img_path(base_path, filename), extracted_tables_dict['EasyOCR'], (30,15))
    for ocr_name, ext in extracted_tables_dict.items():
        print('-'*165)
        print('{:-^165}'.format(' ' + ocr_name + ' '))
        print('-'*165)
        for idx, table in enumerate(ext):
            display_html(table.html_repr(title=f"Таблица {idx + 1}"), raw=True)

## *borderless_tables, confidence=50*

In [ ]:
# выборка примеров
samples = []
for j in [sample_dict[i][10:13] for i in sample_dict.keys()]:
    for k in j:
        samples.append(k)

# Названия и модели OCR
ocr_dict = {
    'EasyOCR':EasyOCR(lang=['ru',"en"]),
    'tesseract':TesseractOCR(lang="rus+eng+kaz"),
}

# Словарь для результатов OCR
extracted_tables_files_dict = dict()

for idx, target_file in enumerate(samples):
    extracted_tables_dict = dict()

    # Открываем документ
    doc = Image(target_file)

    print(idx)

    # Вытаскиваем таблицы из изображения
    for ocr_name, ocr_m in ocr_dict.items():
        print(ocr_name)
        with timex():
            extracted_tables = doc.extract_tables(ocr=ocr_m,
                                                implicit_rows=False,
                                                borderless_tables=True,
                                                min_confidence=50)

            extracted_tables_dict[ocr_name] = extracted_tables
        print()

    clear_output()

    extracted_tables_files_dict[target_file[-8:]] = extracted_tables_dict

clear_output()

# Печатаем
for filename, extracted_tables_dict in extracted_tables_files_dict.items():
    print('{:*^165}'.format(f' File {filename} | Bounding Boxes '))
    print_img_with_bboxes(get_img_path(base_path, filename), extracted_tables_dict['EasyOCR'], (30,15))
    for ocr_name, ext in extracted_tables_dict.items():
        print('-'*165)
        print('{:-^165}'.format(' ' + ocr_name + ' '))
        print('-'*165)
        for idx, table in enumerate(ext):
            display_html(table.html_repr(title=f"Таблица {idx + 1}"), raw=True)

## *borderless_tables, implicit_rows, confidence=25*

In [ ]:
# выборка примеров
samples = []
for j in [sample_dict[i][10:13] for i in sample_dict.keys()]:
    for k in j:
        samples.append(k)

# Названия и модели OCR
ocr_dict = {
    'EasyOCR':EasyOCR(lang=['ru',"en"]),
    'tesseract':TesseractOCR(lang="rus+eng+kaz"),
}

# Словарь для результатов OCR
extracted_tables_files_dict = dict()

for idx, target_file in enumerate(samples):
    extracted_tables_dict = dict()

    # Открываем документ
    doc = Image(target_file)

    print(idx)

    # Вытаскиваем таблицы из изображения
    for ocr_name, ocr_m in ocr_dict.items():
        print(ocr_name)
        with timex():
            extracted_tables = doc.extract_tables(ocr=ocr_m,
                                                implicit_rows=True,
                                                borderless_tables=True,
                                                min_confidence=25)

            extracted_tables_dict[ocr_name] = extracted_tables
        print()

    clear_output()

    extracted_tables_files_dict[target_file[-8:]] = extracted_tables_dict

clear_output()

# Печатаем
for filename, extracted_tables_dict in extracted_tables_files_dict.items():
    print('{:*^165}'.format(f' File {filename} | Bounding Boxes '))
    print_img_with_bboxes(get_img_path(base_path, filename), extracted_tables_dict['EasyOCR'], (30,15))
    for ocr_name, ext in extracted_tables_dict.items():
        print('-'*165)
        print('{:-^165}'.format(' ' + ocr_name + ' '))
        print('-'*165)
        for idx, table in enumerate(ext):
            display_html(table.html_repr(title=f"Таблица {idx + 1}"), raw=True)

## *confidence=10*

In [ ]:
# выборка примеров
samples = []
for j in [sample_dict[i][10:13] for i in sample_dict.keys()]:
    for k in j:
        samples.append(k)

# Названия и модели OCR
ocr_dict = {
    'EasyOCR':EasyOCR(lang=['ru',"en"]),
    'tesseract':TesseractOCR(lang="rus+eng+kaz"),
}

# Словарь для результатов OCR
extracted_tables_files_dict = dict()

for idx, target_file in enumerate(samples):
    extracted_tables_dict = dict()

    # Открываем документ
    doc = Image(target_file)

    print(idx)

    # Вытаскиваем таблицы из изображения
    for ocr_name, ocr_m in ocr_dict.items():
        print(ocr_name)
        with timex():
            extracted_tables = doc.extract_tables(ocr=ocr_m,
                                                implicit_rows=False,
                                                borderless_tables=False,
                                                min_confidence=10)

            extracted_tables_dict[ocr_name] = extracted_tables
        print()

    clear_output()

    extracted_tables_files_dict[target_file[-8:]] = extracted_tables_dict

clear_output()

# Печатаем
for filename, extracted_tables_dict in extracted_tables_files_dict.items():
    print('{:*^165}'.format(f' File {filename} | Bounding Boxes '))
    print_img_with_bboxes(get_img_path(base_path, filename), extracted_tables_dict['EasyOCR'], (30,15))
    for ocr_name, ext in extracted_tables_dict.items():
        print('-'*165)
        print('{:-^165}'.format(' ' + ocr_name + ' '))
        print('-'*165)
        for idx, table in enumerate(ext):
            display_html(table.html_repr(title=f"Таблица {idx + 1}"), raw=True)

## *confidence=5, borderless_tables*

In [ ]:
# выборка примеров
samples = []
for j in [sample_dict[i][10:13] for i in sample_dict.keys()]:
    for k in j:
        samples.append(k)

# Названия и модели OCR
ocr_dict = {
    'EasyOCR':EasyOCR(lang=['ru',"en"]),
    'tesseract':TesseractOCR(lang="rus+eng+kaz"),
}

# Словарь для результатов OCR
extracted_tables_files_dict = dict()

for idx, target_file in enumerate(samples):
    extracted_tables_dict = dict()

    # Открываем документ
    doc = Image(target_file)

    print(idx)

    # Вытаскиваем таблицы из изображения
    for ocr_name, ocr_m in ocr_dict.items():
        print(ocr_name)
        with timex():
            extracted_tables = doc.extract_tables(ocr=ocr_m,
                                                implicit_rows=False,
                                                borderless_tables=True,
                                                min_confidence=5)

            extracted_tables_dict[ocr_name] = extracted_tables
        print()

    clear_output()

    extracted_tables_files_dict[target_file[-8:]] = extracted_tables_dict

clear_output()

# Печатаем
for filename, extracted_tables_dict in extracted_tables_files_dict.items():
    print('{:*^165}'.format(f' File {filename} | Bounding Boxes '))
    print_img_with_bboxes(get_img_path(base_path, filename), extracted_tables_dict['EasyOCR'], (30,15))
    for ocr_name, ext in extracted_tables_dict.items():
        print('-'*165)
        print('{:-^165}'.format(' ' + ocr_name + ' '))
        print('-'*165)
        for idx, table in enumerate(ext):
            display_html(table.html_repr(title=f"Таблица {idx + 1}"), raw=True)

# **table recognition .PNG (PaddleOCR)**
> **Библиотека**: PaddleOCR.

> **Проверка OCR**: PaddleOCR.

> **Результаты**:
* Находит боксы, но не везде, и не очень точно на наших данных;
* Путается в типах распознанных боксах (text, title, table, table_caption, reference);
* Ограниченное количество языков для функции распознавания таблиц (en - английский, ch - китайский), и отстутвие возможности распознавать несколько языков за раз.
* Есть возможность дообучения модели нахождения боксов.
* Вывод таблиц в html, который можно конвертировать в датфрейм.
* Если дообучить модель распознавания боксов + использовать дообученный OCR, то может нам подойти. Или можно попытатся вытащить код распознания таблиц и использовать только его.
* Время на обработку одного изображения (нахождение таблиц и распознание текста в них) 30-50 секунд, в зависимости от количества найденных боксов.



In [ ]:
# !pip install "paddleocr>=2.6.0.3"

In [ ]:
!pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple

In [ ]:
import os
import cv2
from paddleocr import PPStructure,draw_structure_result,save_structure_res
from PIL import Image
from IPython.display import display_html, display

In [ ]:
# Обнаружение таблиц
table_engine = PPStructure(layout=True, show_log=True)

save_folder = '/content/output'
img_path = sample_dict['проверка таблиц'][0]
img = cv2.imread(img_path)
result = table_engine(img)
save_structure_res(result, save_folder, os.path.basename(img_path).split('.')[0])

for line in result:
    line.pop('img')
    print(line)

In [ ]:
# Отображение результатов
result_path = '/content/result.jpg'

font_path = '/content/doc_fonts_simfang.ttf'
image = Image.open(img_path).convert('RGB')
im_show = draw_structure_result(image, result, font_path)
im_show = Image.fromarray(im_show)
im_show.save(result_path)

In [ ]:
print_img(result_path, figsize=(32,32))

### Больше примеров с выводом распознанных таблиц.
> *На изображении не всегда нарисована распознанная таблица.*

In [ ]:
# Обнаружение таблиц
table_engine = PPStructure(layout=True, show_log=False)

save_folder = '/content/output'

for i in range(1,30):
    img_path = sample_dict['проверка таблиц'][i]
    img = cv2.imread(img_path)
    with timex():
        result = table_engine(img)
    save_structure_res(result, save_folder, os.path.basename(img_path).split('.')[0])

    # Отображение результатов
    result_path = '/content/result.jpg'

    font_path = '/content/doc_fonts_simfang.ttf'
    image = Image.open(img_path).convert('RGB')
    im_show = draw_structure_result(image, result, font_path)
    im_show = Image.fromarray(im_show)
    im_show.save(result_path)

    print_img(result_path, figsize=(32,32), title=img_path[-8:])

    # Отображение значениий боксов
    for line_i, line in enumerate(result):
        line.pop('img')
        print(line_i, '-', line)

    # Отображение датафреймов, если они есть
    for i, box_values in enumerate(result):
        if 'html' in box_values['res']:
            print('{:-^150}'.format(' Бокс №' + str(i) + ' '))
            display_html(pd.read_html(box_values['res']['html'])[0])

# **table recognition (PNG, CascadeTabNet)**
> **Результаты**: Не удолось запустить предобученную модель для распознования таблиц. Проблема с обновлёнными библиотеками. Пришлось находить нужные версии библиотек, т.к. за 2 года они успели обновится, а старые версии удалили, но с обновлением также удалили нужные функции для работы/проверки сети, из-за чего не удалось проверить модель.

In [ ]:
# # install dependencies: (use cu101 because colab has CUDA 10.1)
# !pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# # install mmcv-full thus we could use CUDA operators
# !pip install mmcv-full

# # Install mmdetection
# !rm -rf mmdetection
# !git clone https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection

# !pip install -e .

# # install Pillow 7.0.0 back in order to avoid bug in colab
# !pip install Pillow==7.0.0

In [ ]:
!pip install torch torchvision torchaudio
!pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.1/index.html
!pip install terminaltables

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .
%cd "/content"

!pip install -r "/content/mmdetection/requirements/optional.txt"
!python setup.py install
!python setup.py develop
!pip install -r {"/content/mmdetection/requirements.txt"}
!pip install pillow

In [ ]:
!git clone https://github.com/DevashishPrasad/CascadeTabNet.git

In [ ]:
# Download the Pretrained Model
!gdown "https://drive.google.com/u/0/uc?id=1-QieHkR1Q7CXuBu4fp3rYrvDG9j26eFT"

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
# import mmdet
import mmdetection.mmdet as mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In [ ]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot

In [ ]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv

In [ ]:
import mmcv

In [ ]:
# Load model
config_file = '/content/CascadeTabNet/Config/cascade_mask_rcnn_hrnetv2p_w32_20e.py'
checkpoint_file = '/content/epoch_36.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# Test a single image
img = "/content/CascadeTabNet/Demo/demo.png"

# Run Inference
result = inference_detector(model, img)

# Visualization results
show_result_pyplot(img, result,('Bordered', 'cell', 'Borderless'), score_thr=0.85)